GPU attempt. Must be run in wsl

In [ ]:
#data 
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

np.random.seed(1234)
df = pd.read_csv('cancer_dataset.csv')

#convert bool to int
df['test'] = df['test'].astype(int)
df['inSlab'] = df['inSlab'].astype(int)
df['belPl'] = df['belPl'].astype(int)
df['outBx'] = df['outBx'].astype(int)
#drop id column
df = df.drop(columns=["id"], errors="ignore")

X = df.drop(columns=["ca"])  # features
y = df["ca"]  # target labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#stage column (I,II,III,IV) needs to be changed into numerical values
if "Stage" in X_train.columns:
    encoder = LabelEncoder()
    X_train["Stage"] = encoder.fit_transform(X_train["Stage"])
    X_test["Stage"] = encoder.transform(X_test["Stage"])


print("training size", X_train.shape, y_train.shape)
print("testing size", X_test.shape, y_test.shape)

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)
feature_importances = rf.feature_importances_
top_features = np.argsort(feature_importances)[::-1][:feature_dim]  # number of qubits must match number of features
X_train_selected = X_train.iloc[:, top_features]
X_test_selected = X_test.iloc[:, top_features]
print("training size", X_train_selected.shape, y_train.shape)
print("testing size", X_test_selected.shape, y_test.shape)

In [ ]:
from qiskit.circuit.library import ZZFeatureMap

feature_dim = 7  #number of qubits
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=1) #reps=2 repeats the feature mapping twice
from qiskit.primitives import Sampler
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit_algorithms.state_fidelities import ComputeUncompute

sampler = Sampler()
state_fidelity = ComputeUncompute(sampler)
quantum_kernel = FidelityQuantumKernel(fidelity=state_fidelity)

# this implementation is for wsl implementation for GPU training. please ignore if not wsl
from qiskit import transpile
from qiskit_aer import AerSimulator
from qiskit_aer.primitives import Sampler
from qiskit_machine_learning.algorithms import QSVC
from qiskit_machine_learning.kernels import FidelityStatevectorKernel

sim = AerSimulator(method="statevector", device='GPU') # might not need this line
gpu_sampler = Sampler(backend_options={"method": "statevector"})
feature_map = ZZFeatureMap(feature_dimension=feature_dim, reps=1)
gpu_quantum_kernel = FidelityStatevectorKernel(feature_map=feature_map, sampler=gpu_sampler)

#creating our qsvc instance
qsvc = QSVC(quantum_kernel=gpu_quantum_kernel)
print("fitting model")
# Final QSVC Fit
qsvc.fit(X_train_scaled, y_train_scaled)
y_pred_qsvc = qsvc.predict(X_test_scaled)
qsvc_accuracy = accuracy_score(y_test,y_pred_qsvc)